In [6]:
using Printf
using JSON

In [43]:
FUNCTIONS_FILE_PATH = joinpath(
    @__DIR__,
    "functions.json",
)

OUTPUT_TMP_FILE_PATH = joinpath(
    @__DIR__,
    "out.tmp.txt"
)

from_json = JSON.parsefile(
    FUNCTIONS_FILE_PATH
)

step_sizes = from_json["usedStepsizes"]
items = from_json["functions"] |> keys |> collect |> sort;

In [3]:
format_str = raw"""\begin{table}[H]
    \centering
    \resizebox{\textwidth}{!}{
        \begin{tabular}{ll|lllllllll}
        \hline
        \csvreader[
            column count=11,
            no head,
            table head=\hline,
            late after line=\\\hline
        ]
        {results/{{item}}_h_{{stepsize}}.csv}
        {1=\x,2=\ValorReal,3=\ValorAvancada,4=\ErroAvancada,5=\ErroAvancadaRelativa,6=\ValorAtrasada,7=\ErroAtrasada,8=\ErroAtrasadaRelativa,9=\ValorCentrada,10=\ErroCentrada,11=\ErroCentradaRelativa}
        {\x&\ValorReal&\ValorAvancada&\ErroAvancada&\ErroAvancadaRelativa&\ValorAtrasada&\ErroAtrasada&\ErroAtrasadaRelativa&\ValorCentrada&\ErroCentrada&\ErroCentradaRelativa}
    \end{tabular}}
    \caption{Item ${{item}}$, com $h={{stepsize}}$}
\end{table}

\begin{figure}[H]
    \includegraphics[width=\linewidth]{results/{{item}}_h_{{stepsize}}.png}
    \caption{Gráfico plotado para o item ${{item}}$ quando $h={{stepsize}}$}
\end{figure}"""

"\\begin{table}[H]\n    \\centering\n    \\resizebox{\\textwidth}{!}{\n        \\begin{tabular}{ll|lllllllll}\n        \\hline\n        \\csvreader[\n            column count=11,\n            no head,\n            table head=\\hline,\n            late after line=\\\\\\hline\n        ]\n       " ⋯ 414 bytes ⋯ "a}\n    \\end{tabular}}\n    \\caption{Item \${{item}}\$, com \$h={{stepsize}}\$}\n\\end{table}\n\n\\begin{figure}[H]\n    \\includegraphics[width=\\linewidth]{results/{{item}}_h_{{stepsize}}.png}\n    \\caption{Gráfico plotado para o item \${{item}}\$ quando \$h={{stepsize}}\$}\n\\end{figure}"

In [40]:
latex_final_string = ""
for item in items
    ssection_str = ""
    ssection = @sprintf "\\subsection{Item \$%s\$}" item
    ssection_str = ssection_str * ssection * "\n"
    for step in step_sizes
        sssection_str = ""
        sssection = @sprintf "\\subsubsection{\$h=%s\$}" step
        sssection_str = sssection_str * sssection * "\n"
        table_and_figure = replace(format_str, "{{item}}" => item, "{{stepsize}}" => step)
        sssection_str = sssection_str * table_and_figure * "\n"
        ssection_str = ssection_str * sssection_str * "\n"
    end
    latex_final_string = latex_final_string * ssection_str * "\n"
end

In [44]:
open(OUTPUT_TMP_FILE_PATH, "w") do fileio
    print(fileio, latex_final_string)
end